### Dataset sobre carros elétricos

Esse dataset foi retirado diretamente do Kaggle. Seu objetivo é principalmente retirar insights aleatórios utilizando mecânicas do Pandas. As informações variam entre maior quantidade de modelos, estado com o maior número de carros registrados, autonomia do veículo entre outros dados.

Por se tratar de um dataset de exemplo, alguns detalhes podem acabar não sendo bem trabalhados.

#### DataFrames

**df_elec_cars_full** - Dados brutos com NaNs inclusos  
**df_auto_cars** - DataFrame derivado de *df_elec_cars_full* para um agrupamento baseado na Marca dos carros 

In [65]:
import pandas as pd

df_elec_cars_full = pd.read_csv("../data/data.csv")

In [66]:
df_elec_cars_full.head(7)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,E.V_Type,CAFV,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,JTMAB3FV3P,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,PHEV,known,42.0,0.0,35.0,240684006,POINT (-122.8728334 47.5798304),PUGET SOUND ENERGY INC,5.303509e+10
1,1N4AZ1CP6J,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,BEV,known,151.0,0.0,35.0,474183811,POINT (-122.6961203 47.5759584),PUGET SOUND ENERGY INC,5.303508e+10
2,5YJ3E1EA4L,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,BEV,known,266.0,0.0,43.0,113120017,POINT (-122.3340795 47.6099315),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,1N4AZ0CP8E,King,Seattle,WA,98125.0,2014,NISSAN,LEAF,BEV,known,84.0,0.0,46.0,108188713,POINT (-122.304356 47.715668),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
4,1G1FX6S00H,Thurston,Yelm,WA,98597.0,2017,CHEVROLET,BOLT EV,BEV,known,238.0,0.0,20.0,176448940,POINT (-122.5715761 46.9095798),PUGET SOUND ENERGY INC,5.306701e+10
5,5YJYGDEE5L,Snohomish,Lynnwood,WA,98036.0,2020,TESLA,MODEL Y,BEV,known,291.0,0.0,21.0,124511187,POINT (-122.287143 47.812199),PUGET SOUND ENERGY INC,5.306105e+10
6,KM8S6DA23N,Kitsap,Poulsbo,WA,98370.0,2022,HYUNDAI,SANTA FE,PHEV,known,31.0,0.0,23.0,212217764,POINT (-122.6368884 47.7469547),PUGET SOUND ENERGY INC,5.303509e+10


In [67]:
# Trabalhando com valores nulos dentro do DataSet

# List comprehension para criar uma lista apenas com as colunas que possuem valores nulos

nan_columns = [columns for columns in df_elec_cars_full.columns if df_elec_cars_full[columns].isna().sum() > 0]

for item in nan_columns:
    print(f"{item}:", df_elec_cars_full[item].isna().sum())

County: 3
City: 3
Postal Code: 3
Model: 1
Electric Range: 8
Base MSRP: 8
Legislative District: 442
Vehicle Location: 8
Electric Utility: 3
2020 Census Tract: 3


In [68]:
# Verificando como os NaN estão distribuídos. Muitos estão nas mesmas linhas
# Sem dados de Cidades ou Condados/Vilarejos/Distritos eu não tenho como trabalhar de forma efetiva
# Apenas em casos onde eu não utilizo esses dados faltantes vou utilizar o _full

df_elec_cars_full[df_elec_cars_full["County"].isna()]

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,E.V_Type,CAFV,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
138102,WBAJA9C50K,NaN,NaN,AE,NaN,2019,BMW,530E,PHEV,not eligible,16.0,53400.0,NaN,244582593,NaN,NaN,NaN
182254,5YJXCAE24H,NaN,NaN,BC,NaN,2017,TESLA,MODEL X,BEV,known,200.0,0.0,NaN,159850029,NaN,NaN,NaN
203466,1G1RB6S53J,NaN,NaN,BC,NaN,2018,CHEVROLET,VOLT,PHEV,known,53.0,0.0,NaN,477613216,NaN,NaN,NaN


In [69]:
# Criando o DF com os elementos NaN droppados

df_elec_cars_drop = df_elec_cars_full.dropna(subset=["County", "City", "Legislative District"], how='all')

In [70]:
# Checando se ainda existem NaNs nos dados principais que vou utilizar

for item in nan_columns:
    print(f"{item}:", df_elec_cars_drop[item].isna().sum())

County: 0
City: 0
Postal Code: 0
Model: 1
Electric Range: 8
Base MSRP: 8
Legislative District: 439
Vehicle Location: 5
Electric Utility: 0
2020 Census Tract: 0


In [71]:
# Dropando linhas com colunas NaN essenciais e que não podem ser utilizadas nas informações de modelo
# Dropando colunas que não irei utilizar e possuem dados faltantes
# Preenchendo dados nulos onde as colunas possuem dados numéricos que posso trabalhar

df_elec_cars_drop = df_elec_cars_drop.dropna(subset="Model").fillna({
                                                "Electric Range":df_elec_cars_drop["Electric Range"].mean(),
                                                "Base MSRP":df_elec_cars_drop["Base MSRP"].mean()
}).drop(["Legislative District", "Vehicle Location"], axis=1)

In [72]:
df_elec_cars_drop.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,E.V_Type,CAFV,Electric Range,Base MSRP,DOL Vehicle ID,Electric Utility,2020 Census Tract
0,JTMAB3FV3P,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,PHEV,known,42.0,0.0,240684006,PUGET SOUND ENERGY INC,5.303509e+10
1,1N4AZ1CP6J,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,BEV,known,151.0,0.0,474183811,PUGET SOUND ENERGY INC,5.303508e+10
2,5YJ3E1EA4L,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,BEV,known,266.0,0.0,113120017,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,1N4AZ0CP8E,King,Seattle,WA,98125.0,2014,NISSAN,LEAF,BEV,known,84.0,0.0,108188713,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
4,1G1FX6S00H,Thurston,Yelm,WA,98597.0,2017,CHEVROLET,BOLT EV,BEV,known,238.0,0.0,176448940,PUGET SOUND ENERGY INC,5.306701e+10


In [73]:
for item in nan_columns:
    try:
        print(f"{item}:", df_elec_cars_drop[item].isna().sum())
    except:
        print(f"Coluna '{item}' não presente no DataFrame")

County: 0
City: 0
Postal Code: 0
Model: 0
Electric Range: 0
Base MSRP: 0
Coluna 'Legislative District' não presente no DataFrame
Coluna 'Vehicle Location' não presente no DataFrame
Electric Utility: 0
2020 Census Tract: 0


In [74]:
df_elec_cars_full.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205439 entries, 0 to 205438
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   VIN (1-10)            205439 non-null  object 
 1   County                205436 non-null  object 
 2   City                  205436 non-null  object 
 3   State                 205439 non-null  object 
 4   Postal Code           205436 non-null  float64
 5   Model Year            205439 non-null  int64  
 6   Make                  205439 non-null  object 
 7   Model                 205438 non-null  object 
 8   E.V_Type              205439 non-null  object 
 9   CAFV                  205439 non-null  object 
 10  Electric Range        205431 non-null  float64
 11  Base MSRP             205431 non-null  float64
 12  Legislative District  204997 non-null  float64
 13  DOL Vehicle ID        205439 non-null  int64  
 14  Vehicle Location      205431 non-null  object 
 15  

In [75]:
df_elec_cars_full.describe()

,Postal Code,Model Year,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,2020 Census Tract
count,205436.000000,205439.000000,205431.000000,205431.000000,204997.000000,2.054390e+05,2.054360e+05
mean,98177.971870,2020.960363,52.164342,922.670532,28.970848,2.277156e+08,5.297704e+10
std,2419.037479,2.989059,88.075859,7761.753602,14.910052,7.205737e+07,1.588435e+09
min,1731.000000,1997.000000,0.000000,0.000000,1.000000,4.469000e+03,1.001020e+09
25%,98052.000000,2019.000000,0.000000,0.000000,17.000000,1.935324e+08,5.303301e+10
50%,98125.000000,2022.000000,0.000000,0.000000,33.000000,2.382368e+08,5.303303e+10
75%,98372.000000,2023.000000,48.000000,0.000000,42.000000,2.618718e+08,5.305307e+10
max,99577.000000,2025.000000,337.000000,845000.000000,49.000000,4.792548e+08,5.602100e+10


In [76]:
for columns in df_elec_cars_full.columns:
    print(df_elec_cars_full[columns].value_counts())

VIN (1-10)
7SAYGDEE6P    1221
7SAYGDEE7P    1217
7SAYGDEEXP    1181
7SAYGDEE5P    1176
7SAYGDEE8P    1174
              ... 
YV4BC0PM5G       1
YV4ER3GL1R       1
WP0AB2Y12S       1
SALWV2RY9L       1
WA1LAAGE5M       1
Name: count, Length: 12140, dtype: int64
County
King           105237
Snohomish       24721
Pierce          16197
Clark           12231
Thurston         7526
                ...  
Maui                1
Plaquemines         1
Burlington          1
Mobile              1
Christian           1
Name: count, Length: 205, dtype: int64
City
Seattle                    33328
Bellevue                   10235
Redmond                     7341
Vancouver                   7286
Bothell                     6769
                           ...  
Peoria                         1
Winterville                    1
Sarasota                       1
Aberdeen Proving Ground        1
Wishram                        1
Name: count, Length: 770, dtype: int64
State
WA    204997
CA       116
VA        58

In [77]:
# Realizando um agrupamento e um pequeno processo de ETL
# Estou utilizando o df_elec_cars_full pois eu não utilizo as colunas com dados insubstituíveis faltando
# Base MSRP pode ser Substituído por uma média por exemplo

df_auto_cars = df_elec_cars_full.groupby("Make").agg({
                                "Electric Range":"mean",
                                "Make":"count",
                                "Base MSRP":"mean"
}).rename(columns={
                    "Make":"Quantidade",
                    "Electric Range":"Autonomia (KM)"
}).reset_index().rename(columns={
                                "Make":"Marca"
}).sort_values(by=["Autonomia (KM)","Quantidade"], ascending=[False, False])

In [78]:
df_auto_cars.reset_index(drop=True, inplace=True)

In [79]:
df_auto_cars

,Marca,Autonomia (KM),Quantidade,Base MSRP
0,JAGUAR,203.185185,243,0.000000
1,TH!NK,100.000000,5,0.000000
2,WHEEGO ELECTRIC CARS,100.000000,3,32995.000000
3,CHEVROLET,91.017798,15114,0.000000
4,FIAT,83.586984,799,0.000000
5,NISSAN,76.032565,14525,0.000000
6,TESLA,68.234350,90318,1192.710202
7,SMART,61.629032,248,0.000000
8,AZURE DYNAMICS,56.000000,4,0.000000
9,AUDI,46.913987,3918,0.000000


### Realizando uma visão por modelos

Essa visão abaixo é apenas o Modelo dos carros, isso sem perder o acesso da marca dos carros. DataFrame de modelos bem simples realizado com sucesso e aplicação de função.

In [80]:
df_elec_cars_drop[df_elec_cars_drop["Make"] == "TOYOTA"]["Model"].unique()

array(['RAV4 PRIME', 'PRIUS PRIME', 'PRIUS PLUG-IN', 'BZ4X', 'RAV4'],
      dtype=object)

In [81]:
df_elec_cars_drop.columns.to_list()

['VIN (1-10)',
 'County',
 'City',
 'State',
 'Postal Code',
 'Model Year',
 'Make',
 'Model',
 'E.V_Type',
 'CAFV',
 'Electric Range',
 'Base MSRP',
 'DOL Vehicle ID',
 'Electric Utility',
 '2020 Census Tract']

In [82]:
df_models = df_elec_cars_drop.groupby(by="Model").agg({"Make":"max",
                                           "Model":"count",
                                           "Electric Range":"mean",
                                           "Model Year":"max"})

In [83]:
df_models = df_models.rename(columns={"Model":"Quantidade"}).reset_index()

In [84]:
df_models

,Model,Make,Quantidade,Electric Range,Model Year
0,330E,BMW,496,18.157258,2024
1,500,FIAT,780,85.623077,2019
2,500E,FIAT,19,0.000000,2024
3,530E,BMW,430,16.093023,2023
4,740E,BMW,28,14.000000,2019
...,...,...,...,...,...
147,XC40,VOLVO,1118,0.000000,2024
148,XC60,VOLVO,1326,26.713424,2025
149,XC90,VOLVO,1695,23.824189,2025
150,XM,BMW,13,31.000000,2024


In [ ]:
# Os dados de comparação são apenas métodos didáticos para aplicar a função.

def worth_buy(row):
    if row["Electric Range"] > 15:
        return "Worth"
    else:
        return "No worth"

In [92]:
df_models["Worth buy"] = df_models.apply(worth_buy, axis=1)

In [93]:
df_models

,Model,Make,Quantidade,Electric Range,Model Year,Worth buy
0,330E,BMW,496,18.157258,2024,Worth
1,500,FIAT,780,85.623077,2019,Worth
2,500E,FIAT,19,0.000000,2024,No worth
3,530E,BMW,430,16.093023,2023,Worth
4,740E,BMW,28,14.000000,2019,No worth
...,...,...,...,...,...,...
147,XC40,VOLVO,1118,0.000000,2024,No worth
148,XC60,VOLVO,1326,26.713424,2025,Worth
149,XC90,VOLVO,1695,23.824189,2025,Worth
150,XM,BMW,13,31.000000,2024,Worth
